In [1]:
import tensorflow as tf
import numpy as np

In [2]:
value = tf.constant(np.linspace(1,20,20))
value = tf.expand_dims(value,-1)
value = tf.expand_dims(value,0)
#value = tf.expand_dims(value,0)

In [3]:
value.get_shape()

TensorShape([Dimension(1), Dimension(20), Dimension(1)])

In [4]:
filter_ex = tf.ones([3,1,1],tf.float32)

In [5]:
filter_ex.get_shape()

TensorShape([Dimension(3), Dimension(1), Dimension(1)])

In [12]:
with tf.Graph().as_default(), tf.Session() as session:

    def time_to_batch(value, dilation, name=None):
        with tf.name_scope('time_to_batch'):
            shape = tf.shape(value)
            pad_elements = dilation - 1 - (shape[1] + dilation - 1) % dilation
            padded = tf.pad(value, [[0, 0], [0, pad_elements], [0, 0]])
            reshaped = tf.reshape(padded, [-1, dilation, shape[2]])
            transposed = tf.transpose(reshaped, perm=[1, 0, 2])
            return tf.reshape(transposed, [shape[0] * dilation, -1, shape[2]])


    def batch_to_time(value, dilation, name=None):
        with tf.name_scope('batch_to_time'):
            shape = tf.shape(value)
            prepared = tf.reshape(value, [dilation, -1, shape[2]])
            transposed = tf.transpose(prepared, perm=[1, 0, 2])
            return tf.reshape(transposed,
                              [tf.div(shape[0], dilation), -1, shape[2]])


    def causal_conv(value, filter_, dilation, name='causal_conv'):
        with tf.name_scope(name):
            filter_width = tf.shape(filter_)[0]
            print value.get_shape()
            print filter_.get_shape()
            if dilation > 1:
                transformed = time_to_batch(value, dilation)
                print transformed.get_shape()
                conv = tf.nn.conv1d(transformed, filter_, stride=1,
                                    padding='VALID')
                restored = batch_to_time(conv, dilation)
            else:
                restored = tf.nn.conv1d(value, filter_, stride=1, padding='SAME')
            # Remove excess elements at the end.
            out_width = tf.shape(value)[1] - (filter_width - 1) * dilation
            result = tf.slice(restored,
                              [0, 0, 0],
                              [-1, out_width, -1])
            return result
        
    def my_causal_conv(value, filter_, dilation, name='my_causal_conv'):
        with tf.name_scope(name):
            # assume dilation is 1
            filter_width = filter_.get_shape().as_list()[0]
            print filter_width
            if filter_width%2 == 0:
                mask_start = filter_width/2
            else:
                mask_start = filter_width/2 + 1
            mask = np.ones(filter_.get_shape(),np.float32)
            print type(mask)
            mask[mask_start:filter_width,:,:] = 0
            filter_ = tf.multiply(filter_,mask)
            result = tf.nn.conv1d(value,filter_,stride=1,padding='SAME')
            return result
            
        
    value_ex = tf.constant(np.linspace(1,20,20),tf.float32)
    value_ex = tf.expand_dims(value_ex,-1)
    value_ex = tf.expand_dims(value_ex,0)
    filter_ex = tf.ones([3,1,1],tf.float32)
    
    res = causal_conv(value_ex,filter_ex,1)
    res2 = my_causal_conv(value_ex,filter_ex,1)
    
    result,result2,value_ex = session.run([res,res2,value_ex])
    print result.shape
    print result
    print result2.shape
    print result2
    print value_ex

(1, 20, 1)
(3, 1, 1)
3
<type 'numpy.ndarray'>
(1, 18, 1)
[[[  3.]
  [  6.]
  [  9.]
  [ 12.]
  [ 15.]
  [ 18.]
  [ 21.]
  [ 24.]
  [ 27.]
  [ 30.]
  [ 33.]
  [ 36.]
  [ 39.]
  [ 42.]
  [ 45.]
  [ 48.]
  [ 51.]
  [ 54.]]]
(1, 20, 1)
[[[  1.]
  [  3.]
  [  5.]
  [  7.]
  [  9.]
  [ 11.]
  [ 13.]
  [ 15.]
  [ 17.]
  [ 19.]
  [ 21.]
  [ 23.]
  [ 25.]
  [ 27.]
  [ 29.]
  [ 31.]
  [ 33.]
  [ 35.]
  [ 37.]
  [ 39.]]]
[[[  1.]
  [  2.]
  [  3.]
  [  4.]
  [  5.]
  [  6.]
  [  7.]
  [  8.]
  [  9.]
  [ 10.]
  [ 11.]
  [ 12.]
  [ 13.]
  [ 14.]
  [ 15.]
  [ 16.]
  [ 17.]
  [ 18.]
  [ 19.]
  [ 20.]]]


In [ ]:
def causual_conv1d(value, filters, )

In [ ]:
def artrous_conv1d(value, filters, rate, padding, name=None):
    '''
    1d equivalent of the tensorflow artrous 2d convolution

    Args:
        value: input 3-D tensor with shape [batch_size, in_width, ip_channels]
        filters: convolution filter with shape [filter_width, ip_channels, op_channels]
        rate: rate of dilation, 1 is equivalent to normal convolution
        padding: string, "SAME" or "VALID" only
        name: function scoping name for tensorflow

    Returns:
    Tensor of shape [batch_size, new_width, op_channels]
    '''
    with tf.name_scope(name, "artrous_conv1d") as name:
        if rate == 1:
            # perform normal convolution
            return tf.nn.conv1d(value,filters,stride=1,padding=padding)

        elif rate > 1:
            # determine padding based on padding choice
            if padding == "SAME":
                # based on the size of the kernel add zeroes to the image
                filter_width = tf.shape(filters)[0]
                overall_pad = filter_width + filter_width*(rate-1)
                pad_left = overall_pad//2
                pad_right = overall_pad - pad_left

            else:
                pad_left = 0
                pad_right = 0

            # check optimality with the rate provided
            check_width = pad_left + tf.shape(value) + pad_right
            pad_right_extra = (rate - check_width % rate) % rate
            pad_right += pad_right_extra

            # reshape value tensor based on type of padding and rate
            new_value = space_to_batch_1d(value,[pad_left,pad_right],rate)

            # perform 1d convolution
            conv_new_value = tf.nn.conv1d(new_value,filters,stride=1,padding='VALID')

            # re-shape the output to output size
            conv_value = batch_to_space_1d(conv_new_value,[0,pad_right_extra],rate)

            return conv_value

        else:
            raise ValueError("Rate must be >= 1")